[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/basf/mlipx/blob/main/docs/source/notebooks/combine.ipynb)

# Combine Multiple Nodes

The `mlipx` command line interface provides you with recipes for specific tasks.
In this notebook, we will write a script to include different aspects from different recipes into a single workflow.

In [ ]:
# Only install the packages if they are not already installed
!pip show mlipx > /dev/null 2>&1 || pip install mlipx
!pip show rdkit2ase > /dev/null 2>&1 || pip install rdkit2ase

In [ ]:
# We will create a GIT and DVC repository in a temporary directory
import os
import tempfile

temp_dir = tempfile.TemporaryDirectory()
os.chdir(temp_dir.name)

Like all `mlipx` Nodes we will use a GIT and DVC repository to run experiments.
To make our custom code available, we structure our project like

```
relaxation/
   ├── .git/
   ├── .dvc/
   ├── src/__init__.py
   ├── src/relaxation.py
   ├── models.py
   └── main.py
```

to allow us to import our code `from src.relaxation import Relax`.
Alternatively, you can package your code and import it like any other Python package.

In [ ]:
!git init
!dvc init --quiet

Let us configure now configure a workflow, creating a structure from `SMILES`, relax it, run molecular dynamics and compute the homonuclear diatomics.

In [ ]:
import mlipx

project = mlipx.Project()

emt = mlipx.GenericASECalculator(
    module="ase.calculators.emt",
    class_name="EMT",
)

with project.group("initialize"):
    confs = mlipx.Smiles2Conformers(smiles="CCCC", num_confs=1)

with project.group("structure-optimization"):
    struct_optim = mlipx.StructureOptimization(
        data=confs.frames, data_id=-1, optimizer="LBFGS", model=emt
    )

thermostat = mlipx.LangevinConfig(
    timestep=0.5,
    temperature=300,
    friction=0.001,
)

with project.group("molecular-dynamics"):
    md = mlipx.MolecularDynamics(
        data=struct_optim.frames,
        data_id=-1,
        model=emt,
        thermostat=thermostat,
        steps=1000,
    )

with project.group("homonuclear-diatomics"):
    ev = mlipx.HomonuclearDiatomics(
        data=confs.frames,
        model=emt,
        n_points=100,
        min_distance=0.75,
        max_distance=2.0,
        elements=[],
    )

project.repro()

Once the graph has been executed, we can look at the resulting structures.

In [ ]:
struct_optim.figures["energy_vs_steps"]

In [ ]:
md.figures["energy"]

In [ ]:
print(ev.figures.keys())
ev.figures["C-C bond"]

In [ ]:
temp_dir.cleanup()